# Лабораторын ажил 9

## 0. Хэрэгцээтэй сан болон өгөгдлөө бэлтгэх

In [50]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable


In [51]:
!pip show scikit-learn

Name: scikit-learn
Version: 0.22.2.post1
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: None
Author-email: None
License: new BSD
Location: /home/bmmn/.local/lib/python3.6/site-packages
Requires: scipy, joblib, numpy
Required-by: 


In [48]:
import pandas as pd
import re
import string
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
nltk.download('punkt')
import numpy as np
from nltk.tokenize import word_tokenize
import csv

[nltk_data] Downloading package punkt to /home/bmmn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 0.1. Өгөгдлөө уншиж авах

In [4]:
csv_file = "dataset/newsdata - newsdata.csv"


In [5]:
datalist = pd.read_csv(csv_file)

In [6]:
df = pd.DataFrame(datalist)


In [7]:
df.head()

,news,label
0,"Хятадын эрдэмтэд Европ, Америкийн зүүн эргээр ...",Дэлхий
1,Францад есөн настай сурагч хүү хоёрдугаар сард...,Дэлхий
2,Москвад гуравдугаар сарын дунд үед баригдаж эх...,Дэлхий
3,Хойд Солонгосын удирдагч Ким Жон Ун мэс засалд...,Дэлхий
4,ОХУ-д “covid-19” өвчинтэй хүмүүсийг эмчилж бай...,Дэлхий


In [8]:
df.loc[5]['news']

'Оксфорд их сургуулийн эрдэмтдийн гарган авсан коронавирусийн эсрэг вакциныг маргаашаас (2020.4.23) эхлэн хүн дээр турших талаар Их Британийн Эрүүл мэндийн сайд Матт Ханкок мэдэгджээ. Их сургуулийн эрдэмтдийн багийн гишүүдийн мэдээлснээр вакцины туршилт амжилттай болбол ирэх есдүгээр сараас олон сая тунгаар үйлдвэрлэж эхлэх боломжтой гэж тооцжээ. Коронавирусийн халдварт өвчний эсрэг вакцин гаргахаар олон орны эрдэмтэд цагтай уралдан ажиллаж байгаа ч өнөөдрийн байдлаар олон улсад 175 мянга гаруй хүн хэдийн нас барж, эдийн засгийн том хямрал үүсгээд байна. Их Британийн эрүүл мэндийн сайд хэвлэлийн хурлын үеэр мэдэгдэхдээ вакцин гарган авах ажилд Засгийн газраас гол анхаарлаа хандуулж, Оксфорд их сургуулийн эрдэмтдийн багт 20 сая фунт-стерлингийн санхүүжилт олгожээ. Түүнчлэн Лондон дахь Эзэн хааны коллежийн судалгааны ажилд 22,5 сая фунт-стерлингийг төсөвлөсөн байна. Эх сурвалж: The Independent'

## 1. Текст цэвэрлэх, бэлтгэх

### 1.1. Тусгай тэмдэгт цэвэрлэгээ


In [9]:
df['Content_Parsed_1'] = df['news'].str.replace("\xa0", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\r", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("xa07", "")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("«", "")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("»", "")    
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('(', '')
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace(')', '')
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('”', '')
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('“', '')
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('-', ' ')

### Тоо


In [ ]:
# Хэрэгтэй гэж үзвэл тоонуудыг авч болно.
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("0", "")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("1", "")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("2", "")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("3", "")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("4", "")    
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("5", "")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("6", "")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("7", "")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("8", "")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("9", "")

### 1.2. Том үсгийг жижиг үсэг болгох

In [10]:

df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()

### 1.3. Дээрх бүх үйлдлийг нэг дор хийж болох функц



In [11]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    

    if len(text) > 2:
        return ' '.join(word for word in text.split())
txt = 'молобөрйлоб бйроө (f) ....3.2.1..3.4.!@@#$%^&*(())))))))____+|~``~~|-+*/^%$#@! ?>/.,<:">:;:"'''
print(clean_text(txt))
df['Content_Parsed_3'] = pd.DataFrame(df['Content_Parsed_2'].apply(lambda x: clean_text(x)))


молобөрйлоб бйроө f


### 1.4. Stop words

In [12]:
# Stop words устгах regular expression жишээ
example = "Сайн байна уу?"
word = "уу"

regex = r"\b" + word + r"\b"  

re.sub(regex, "", example)

'Сайн байна ?'

In [14]:
# Stop words-уудаа уншиж авах
import string
stopwords = open("stopwords1.txt","r")
print ("Нэр", stopwords.name)
mnstopwords = stopwords.read().split('\n')


Нэр stopwords1.txt


одоо stop words-ийг ялгаж авах боломжтой.

In [15]:
# Stop words устгах
df['Content_Parsed_4'] = df['Content_Parsed_3']

for stop_word in mnstopwords:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_4'] = df['Content_Parsed_4'].str.replace(regex_stopword, '')

### 1.5. Stemming


1. Хэрвээ хоосон мөр байвал устгана

In [16]:
df = df.mask(df.astype(object).eq('None')).dropna()
df = df.mask(df.astype(object).eq('nan')).dropna()
df = df.mask(df.astype(object).eq('NaN')).dropna()
df= df.reset_index(drop=True)


In [17]:
df['Content_Parsed_4'].head(75660)

0      хятадын эрдэмтэд европ америкийн зүүн эргээр т...
1      францад есөн настай сурагч хүү хоёрдугаар сард...
2      москвад гуравдугаар    баригдаж эхэлсэн халдва...
3      хойд солонгосын удирдагч ким жон ун мэс засалд...
4      оху  covid өвчинтэй хүмүүсийг эмчилж  бөөрний ...
                             ...                        
312     баригч намынхан тун овжин аргатай  аливаа асу...
313    таны амь  албаны ажлыг хурдан чанартай гүйцэтг...
314    британийн загварын зөвлөл хүйсээр ялгасан шоуг...
315    корона­вирусийн халдвараас сэргийлэх үүднээс м...
316    улсын онцгой комиссоос гарсан шийдвэрийн  тусг...
Name: Content_Parsed_4, Length: 317, dtype: object

2. Stemming


Stemming хийж үгийн төгсгөлийг авч хаяна.

In [18]:
ignore_words = ['', '-ийн', '-ын','-т', '-тай', '-г','т', '-ийг','-д', '-н', '-ний', '-дээр', 'ч', 'л', 'нь', 'юм', 'бэ', 'вэ', 'юу', 'гэж', 'уу']
my_rule_tuple = ("аа$|ч$|д$|ий$|ыг$|ы$|ийг$|ны$|наас$|ний$|лаа$|лээ$|лоо$|лөө$|даа$|дээ$|доо$|дөө$|уулж$|үүлж$|гдах$|гдэх$|гдох$|гдөх$|лалт$|лэлт$|лолт$|лөлт$|ахад$|дэхэд$|доход$|дөхөд$|уудыг$|үүдийг$|нүүдийг$|нуудыг$|чихлаа$|чихлээ$|чихлоо$|чихлөө$|ид$|ын$|ийн$|уудын$|үүдийн$|тай$|той$|төй$|тэй$|лал$|лэл$|лол$|лөл$|аар$|ээр$|оор$|өөр$|лах$|лэх$|лох$|лөх$|ахад$|эхэд$|оход$|өхөд$|лалд$|лэлд$|лөлд$|лолд$|ууд$|үүд$|даг$|дэг$|дог$|дөг$|ддаг$|ддэг$|ддог$|ддөг$|даж$|дэж$|дож$|дөж$|сэн$|сан$|сөн$|сон$|гах$|гэх$|гох$|гөх$|рах$|рэх$|рох$|рөх$|иас$|иэс$|иос$|иөс$|нд$|нт$|ааж$|ээж$|оож$|өөж$|лаар$|лээр$|лоор$|лөөр$|эг$|ог$|өг$|уг$|өөг$|ээг$|оог$|үүг$|ууг$|бар$|бэр$|бор$|бөр$|бур$|бүр$|раа$|рээ$|рөө$|рүү$|роо$|руу$|аас$|ээс$|оос$|өөс$|аарх$|ээрх$|оорх$|уурх$|үүрх$|өөрх$|аад$|ээд$|ууд$|үүд$|оод$|өөд$|нууд$|нүүд$|дсан$|дсэн$|дсөн$|дсон$|дах$|дэх$|дох$|дөх$|хлаа$|хлээ$|хлоо$|хлөө$|лахдаа$|лэхдээ$|лохдоо$|лөхдөө$|лдаг$|лдэг$|лж$|аагүй$|ээгүй$|оогүй$|өөгүй$|даггүй$|дэггүй$|доггүй$|дөггүй$|нуудийнхаа$|нүүдийнхээ$|гүй$")
stemmer = stem.RegexpStemmer(my_rule_tuple, min = 4)

def lemmatizer(text):        
    sent = []
    tokens = word_tokenize(text)
    for word in tokens:
      if word not in ignore_words:
        sent.append(stemmer.stem(word))
    return " ".join(sent)
txt = 'шударга өрсөлдөөн хэрэглэгчийн зуун мускг дуу дүрсний чанартай шинэчиллээ улсын улс усаа гараанаас урлаг'
print(lemmatizer(txt))
df['Content_Parsed_5'] =  df.apply(lambda x: lemmatizer(x['Content_Parsed_4']), axis=1)

шударга өрсөлдөөн хэрэглэгч зуун мускг дуу дүрс чанар шинэчил улс улс ус гараа урлаг


### 1.6. Хийсэн зүйлээ нэгтгэн харцгаая

In [19]:
df.loc[6]['news']

'Коронавирусийн халдвараас сэргийлсэн хөл хорионы арга хэмжээнээс шалтгаалан ОХУ-ын дээд шүүх хуралдаанаа онлайнаар явуулахаар шийдвэрлэжээ. Энэ нь Дээд шүүх хуралдаанаа өөр хэлбэрээр хийж байгаа анхны тохиолдол юм. Эхний ээлжинд иргэний хэргийн маргаантай холбоотой шүүх хурлыг онлайнаар хийхээр болсон байна. Нэхэмжлэгч болон хариуцагч талууд гэрээсээ, эсвэл албан өрөөнөөсөө өөр өөрсдийн электрон төхөөрөмжийг ашиглан оролцох ажээ. Шүүх хуралд оролцогч талууд онлайнд хуралд оролцох төлөөлөгчдийн бүрэн эрхтэйг баталгаажуулсан албан бичиг, мөн ашиглах интернэт урсгал нь баталгаат компанийх гэдгийг нотолсон албан бичгийг Дээд шүүхэд ирүүлэх учиртай. Дээд шүүхийн зүгээс эдгээрийн хүлээн авч нягтлаад онлайн шүүх хуралдаанд оролцох эрх үүссэнийг цахим шуудангаар мэдэгдэх ажээ. Энэ хэлбэр нь өнөөгийн үүссэн нөхцөл байдалтай холбоотойгоор бий болж байгаа ч шүүх хуралд биеэр ирж оролцох боломжгүй иргэдийн хувьд хэрэглэж болох арга байхыг үгүйсгэхгүй хэмээн Дээд шүүх үзэж байна. Эх сурвалж: РИА Н

1. Тусгай тэмдэгт цэвэрлэгээ

In [20]:
df.loc[6]['Content_Parsed_1']

'Коронавирусийн халдвараас сэргийлсэн хөл хорионы арга хэмжээнээс шалтгаалан ОХУ ын дээд шүүх хуралдаанаа онлайнаар явуулахаар шийдвэрлэжээ. Энэ нь Дээд шүүх хуралдаанаа өөр хэлбэрээр хийж байгаа анхны тохиолдол юм. Эхний ээлжинд иргэний хэргийн маргаантай холбоотой шүүх хурлыг онлайнаар хийхээр болсон байна. Нэхэмжлэгч болон хариуцагч талууд гэрээсээ, эсвэл албан өрөөнөөсөө өөр өөрсдийн электрон төхөөрөмжийг ашиглан оролцох ажээ. Шүүх хуралд оролцогч талууд онлайнд хуралд оролцох төлөөлөгчдийн бүрэн эрхтэйг баталгаажуулсан албан бичиг, мөн ашиглах интернэт урсгал нь баталгаат компанийх гэдгийг нотолсон албан бичгийг Дээд шүүхэд ирүүлэх учиртай. Дээд шүүхийн зүгээс эдгээрийн хүлээн авч нягтлаад онлайн шүүх хуралдаанд оролцох эрх үүссэнийг цахим шуудангаар мэдэгдэх ажээ. Энэ хэлбэр нь өнөөгийн үүссэн нөхцөл байдалтай холбоотойгоор бий болж байгаа ч шүүх хуралд биеэр ирж оролцох боломжгүй иргэдийн хувьд хэрэглэж болох арга байхыг үгүйсгэхгүй хэмээн Дээд шүүх үзэж байна. Эх сурвалж: РИА Н

2. Том үсгийг жижиг үсэг болгох

In [21]:
df.loc[6]['Content_Parsed_2']

'коронавирусийн халдвараас сэргийлсэн хөл хорионы арга хэмжээнээс шалтгаалан оху ын дээд шүүх хуралдаанаа онлайнаар явуулахаар шийдвэрлэжээ. энэ нь дээд шүүх хуралдаанаа өөр хэлбэрээр хийж байгаа анхны тохиолдол юм. эхний ээлжинд иргэний хэргийн маргаантай холбоотой шүүх хурлыг онлайнаар хийхээр болсон байна. нэхэмжлэгч болон хариуцагч талууд гэрээсээ, эсвэл албан өрөөнөөсөө өөр өөрсдийн электрон төхөөрөмжийг ашиглан оролцох ажээ. шүүх хуралд оролцогч талууд онлайнд хуралд оролцох төлөөлөгчдийн бүрэн эрхтэйг баталгаажуулсан албан бичиг, мөн ашиглах интернэт урсгал нь баталгаат компанийх гэдгийг нотолсон албан бичгийг дээд шүүхэд ирүүлэх учиртай. дээд шүүхийн зүгээс эдгээрийн хүлээн авч нягтлаад онлайн шүүх хуралдаанд оролцох эрх үүссэнийг цахим шуудангаар мэдэгдэх ажээ. энэ хэлбэр нь өнөөгийн үүссэн нөхцөл байдалтай холбоотойгоор бий болж байгаа ч шүүх хуралд биеэр ирж оролцох боломжгүй иргэдийн хувьд хэрэглэж болох арга байхыг үгүйсгэхгүй хэмээн дээд шүүх үзэж байна. эх сурвалж: риа н

3. Тэмдэгт

In [22]:
df.loc[6]['Content_Parsed_3']

'коронавирусийн халдвараас сэргийлсэн хөл хорионы арга хэмжээнээс шалтгаалан оху ын дээд шүүх хуралдаанаа онлайнаар явуулахаар шийдвэрлэжээ энэ нь дээд шүүх хуралдаанаа өөр хэлбэрээр хийж байгаа анхны тохиолдол юм эхний ээлжинд иргэний хэргийн маргаантай холбоотой шүүх хурлыг онлайнаар хийхээр болсон байна нэхэмжлэгч болон хариуцагч талууд гэрээсээ эсвэл албан өрөөнөөсөө өөр өөрсдийн электрон төхөөрөмжийг ашиглан оролцох ажээ шүүх хуралд оролцогч талууд онлайнд хуралд оролцох төлөөлөгчдийн бүрэн эрхтэйг баталгаажуулсан албан бичиг мөн ашиглах интернэт урсгал нь баталгаат компанийх гэдгийг нотолсон албан бичгийг дээд шүүхэд ирүүлэх учиртай дээд шүүхийн зүгээс эдгээрийн хүлээн авч нягтлаад онлайн шүүх хуралдаанд оролцох эрх үүссэнийг цахим шуудангаар мэдэгдэх ажээ энэ хэлбэр нь өнөөгийн үүссэн нөхцөл байдалтай холбоотойгоор бий болж байгаа ч шүүх хуралд биеэр ирж оролцох боломжгүй иргэдийн хувьд хэрэглэж болох арга байхыг үгүйсгэхгүй хэмээн дээд шүүх үзэж байна эх сурвалж риа новости'

4. Stop words

In [23]:
df.loc[6]['Content_Parsed_4']

'коронавирусийн халдвараас сэргийлсэн хөл хорионы арга хэмжээнээс шалтгаалан оху   шүүх хуралдаанаа онлайнаар явуулахаар шийдвэрлэжээ    шүүх хуралдаанаа  хэлбэрээр   анхны тохиолдол  эхний ээлжинд иргэний хэргийн маргаантай холбоотой шүүх хурлыг онлайнаар хийхээр   нэхэмжлэгч  хариуцагч талууд гэрээсээ эсвэл албан өрөөнөөсөө  өөрсдийн электрон төхөөрөмжийг ашиглан оролцох ажээ шүүх хуралд оролцогч талууд онлайнд хуралд оролцох төлөөлөгчдийн  эрхтэйг баталгаажуулсан албан бичиг  ашиглах интернэт урсгал  баталгаат компанийх  нотолсон албан бичгийг  шүүхэд ирүүлэх учиртай  шүүхийн зүгээс эдгээрийн   нягтлаад онлайн шүүх хуралдаанд оролцох  үүссэнийг цахим шуудангаар мэдэгдэх ажээ  хэлбэр  өнөөгийн үүссэн нөхцөл байдалтай холбоотойгоор     шүүх хуралд биеэр ирж оролцох боломжгүй иргэдийн  хэрэглэж  арга байхыг үгүйсгэхгүй   шүүх үзэж   сурвалж риа новости'

5. Stemming

In [24]:
df.loc[6]['Content_Parsed_5']

'коронавирус халдвар сэргийл хөл хорио арга хэмжээн шалтгаалан оху шүүх хуралдаан онлайн явуулах шийдвэрлэжээ шүүх хуралдаан хэлбэр анх тохиолдол эх ээлжи иргэ хэрг маргаан холбоо шүүх хурл онлайн хийх нэхэмжлэг хариуцаг тал гэрээсээ эсвэл албан өрөөнөөсөө өөрсд электрон төхөөрөмж ашиглан оролцох ажээ шүүх хурал оролцог тал онлай хурал оролцох төлөөлөгчд эрхтэйг баталгаажуул албан бичиг ашиг интернэт урсгал баталгаат компанийх нотол албан бичг шүүхэ ирүү учир шүүх зүг эдгээр нягтл онлайн шүүх хуралдаа оролцох үүссэн цахим шууданг мэдэ ажээ хэл өнөөг үүс нөхцөл байдал холбоотойг шүүх хурал биеэр ирж оролцох боломж иргэд хэрэглэж арга байх үгүйсгэх шүүх үзэж сурва риа новости'

Эцэст нь завсрын багануудыг устгах хэрэгтэй:

In [25]:
df.head(3)

,news,label,Content_Parsed_1,Content_Parsed_2,Content_Parsed_3,Content_Parsed_4,Content_Parsed_5
0,"Хятадын эрдэмтэд Европ, Америкийн зүүн эргээр ...",Дэлхий,"Хятадын эрдэмтэд Европ, Америкийн зүүн эргээр ...","хятадын эрдэмтэд европ, америкийн зүүн эргээр ...",хятадын эрдэмтэд европ америкийн зүүн эргээр т...,хятадын эрдэмтэд европ америкийн зүүн эргээр т...,хятад эрдэмтэ европ америк зүүн эрг тарх корон...
1,Францад есөн настай сурагч хүү хоёрдугаар сард...,Дэлхий,Францад есөн настай сурагч хүү хоёрдугаар сард...,францад есөн настай сурагч хүү хоёрдугаар сард...,францад есөн настай сурагч хүү хоёрдугаар сард...,францад есөн настай сурагч хүү хоёрдугаар сард...,франца е нас сураг хүү хоёрдуг сар коронавирус...
2,Москвад гуравдугаар сарын дунд үед баригдаж эх...,Дэлхий,Москвад гуравдугаар сарын дунд үед баригдаж эх...,москвад гуравдугаар сарын дунд үед баригдаж эх...,москвад гуравдугаар сарын дунд үед баригдаж эх...,москвад гуравдугаар баригдаж эхэлсэн халдва...,москва гуравдуг бариг эхэл халдварт өвч цоо ши...


In [26]:
#Stemming хийгээгүй баганыг авлаа
list_columns = [ "news","label", "Content_Parsed_4"]
df = df[list_columns]

df = df.rename(columns={'Content_Parsed_4': 'news_clean'})

In [27]:
df.head()

,news,label,news_clean
0,"Хятадын эрдэмтэд Европ, Америкийн зүүн эргээр ...",Дэлхий,хятадын эрдэмтэд европ америкийн зүүн эргээр т...
1,Францад есөн настай сурагч хүү хоёрдугаар сард...,Дэлхий,францад есөн настай сурагч хүү хоёрдугаар сард...
2,Москвад гуравдугаар сарын дунд үед баригдаж эх...,Дэлхий,москвад гуравдугаар баригдаж эхэлсэн халдва...
3,Хойд Солонгосын удирдагч Ким Жон Ун мэс засалд...,Дэлхий,хойд солонгосын удирдагч ким жон ун мэс засалд...
4,ОХУ-д “covid-19” өвчинтэй хүмүүсийг эмчилж бай...,Дэлхий,оху covid өвчинтэй хүмүүсийг эмчилж бөөрний ...


## 2. Label кодлох

Сургалт хийхэд label монгол байх нь заримдаа асуудал үүсгэдэг:

In [28]:
label_codes = {
    'Эдийн засаг':'economy',
    'спорт':'sport',
    'улс төр':'politics',
    'Дэлхий':'world',
    'ЭНТЕРТАЙНМЕНТ':'entertainment',
    'Нийгэм':'society',
    'Шар мэдээ':'yellowNews'
}

In [29]:
# Category mapping
df['label_Eng'] = df['label']
df = df.replace({'label_Eng':label_codes})


In [30]:
df.tail()

,news,label,news_clean,label_Eng
312,"Эрх баригч намынхан тун ""овжин"" аргатай нь али...",Шар мэдээ,баригч намынхан тун овжин аргатай аливаа асу...,yellowNews
313,"Таны амь болон албаны ажлыг хурдан, чанартай ...",Шар мэдээ,таны амь албаны ажлыг хурдан чанартай гүйцэтг...,yellowNews
314,Британийн загварын зөвлөл хүйсээр ялгасан шоуг...,Шар мэдээ,британийн загварын зөвлөл хүйсээр ялгасан шоуг...,yellowNews
315,Корона­вирусийн халдвараас сэргийлэх үүднээс М...,Шар мэдээ,корона­вирусийн халдвараас сэргийлэх үүднээс м...,yellowNews
316,Улсын онцгой комиссоос гарсан шийдвэрийн дагуу...,Шар мэдээ,улсын онцгой комиссоос гарсан шийдвэрийн тусг...,yellowNews


### Csv файл руу хадгалах


In [31]:
list_columns = [ "news_clean","label_Eng"]
df = df[list_columns]

In [32]:
df.tail()

,news_clean,label_Eng
312,баригч намынхан тун овжин аргатай аливаа асу...,yellowNews
313,таны амь албаны ажлыг хурдан чанартай гүйцэтг...,yellowNews
314,британийн загварын зөвлөл хүйсээр ялгасан шоуг...,yellowNews
315,корона­вирусийн халдвараас сэргийлэх үүднээс м...,yellowNews
316,улсын онцгой комиссоос гарсан шийдвэрийн тусг...,yellowNews


In [34]:
df.to_csv("dataset/news.mnCleanData.csv",index=False)

## 3. Train - test хэсэгт хуваана

In [35]:
X_train, X_test, y_train, y_test = train_test_split(df['news_clean'], 
                                                    df['label_Eng'], 
                                                    test_size=0.2, 
                                                    random_state=8)

## 4. Текстийн дүрслэл

Бид TF-IDF Векторуудыг шинж чанар болгон ашиглах болно.

Бид өөр өөр параметрүүдийг тодорхойлох ёстой:

* `ngram_range`: Unigram, Bigram, Trigram-ийг авч үзнэ
* `max_df`: Үгсийн сан байгуулахдаа баримт бичгийн давтамжийг өгөгдсөн хэмжээнээс хэт өндөр байх нөхцлийг орхино.
* `min_df`: Үгсийн сан байгуулахдаа баримт бичгийн давтамжийг өгөгдсөн хэмжээнээс доогуур байгаа нөхцлийг орхино.


In [36]:
ngram_range = (1,3)
min_df = 10
max_df = 1.
max_features = 1000

In [37]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(253, 1000)
(64, 1000)


In [38]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(label_codes.items()):
  features_chi2 = chi2(features_train, labels_train == category_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  trigrams = [v for v in feature_names if len(v.split(' ')) == 3]
  print("# '{}' category:".format(Product))
  print("   Хамгийн их хамааралтай unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
  print("   Хамгийн их хамааралтай bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
  print("   Хамгийн их хамааралтай trigrams:\n. {}".format('\n. '.join(trigrams[-2:])))
 
  print("")


# 'Дэлхий' category:
   Хамгийн их хамааралтай unigrams:
. түүнчлэн
. ану
. хүмүүсийн
. халдвараар
. нас
   Хамгийн их хамааралтай bigrams:
. ахуйн нэгж
. хүн нас
   Хамгийн их хамааралтай trigrams:
. улсын онцгой комисст
. халдварт өвчин судлалын

# 'Нийгэм' category:
   Хамгийн их хамааралтай unigrams:
. нийслэлийн
. халдвараас
. амны
. мэдээлэл
. коронавируст
   Хамгийн их хамааралтай bigrams:
. засаг даргын
. эрүүл мэндийн
   Хамгийн их хамааралтай trigrams:
. дүүргийн эрүүгийн хэргийн
. эрүүгийн хэргийн анхан

# 'Шар мэдээ' category:
   Хамгийн их хамааралтай unigrams:
. мэдээлэл
. мэдэхгүй
. ярьжээ
. бичлэг
. эмэгтэй
   Хамгийн их хамааралтай bigrams:
. онцгой комиссын
. ахуйн нэгж
   Хамгийн их хамааралтай trigrams:
. өнгөрсөн долоо хоногт
. улсын онцгой комиссын

# 'ЭНТЕРТАЙНМЕНТ' category:
   Хамгийн их хамааралтай unigrams:
. хоног
. мэдээлэл
. зориг
. ардчилсан
. уран
   Хамгийн их хамааралтай bigrams:
. хэргийн анхан
. анхан шатны
   Хамгийн их хамааралтай trigrams:
. захир

In [39]:
bigrams

['хан уул',
 'хариу арга',
 'урьдчилан сэргийлэх',
 'халдвараас урьдчилан',
 'халдварт өвчин',
 'хуулийн зүйлийн',
 'хуулийн төслийг',
 'хэргийн анхан',
 'хурлын гишүүн',
 'улсын хурлын',
 'тамгын газрын',
 'тусламж үйлчилгээ',
 'сонгуулийн хуулийн',
 'сэргийлэх арга',
 'уих гишүүн',
 'улаанбаатар хотод',
 'улс төр',
 'улс төрд',
 'улс төрийн',
 'улс төрөөс',
 'улсын засгийн',
 'улсын онцгой',
 'явган зам',
 'эмч эмнэлгийн',
 'эрүүгийн хэргийн',
 'эрүүл аюулгүй',
 'эрүүл мэндийн',
 'эхний ээлжинд',
 'өнгөрсөн долоо',
 'өнөөдрийн байдлаар',
 'өссөн хүн',
 'өвчин судлалын',
 'үзэх шалтгаангүйгээр',
 'үндсэн хуулийн',
 'элчин сайдын',
 'хянан шийдвэрлэсэн',
 'хянан шийдвэрлэх',
 'хүн нас',
 'хүндэтгэн үзэх',
 'цар тахал',
 'цар тахлын',
 'шатны шүүх',
 'эдийн засаг',
 'эдийн засгийн',
 'чиг үүрэг',
 'бундармаа нарыг',
 'баянзүрх дүүргийн',
 'ддамба очир',
 'долоо хоногт',
 'гэмт хэргийн',
 'албан ёсны',
 'албанаас мэдээллээ',
 'албан хаагчид',
 'ii iii',
 'албадан ирүүлэх',
 'анхан шатны'

In [40]:
unigrams

['covid',
 'хамтдаа',
 'хамтран',
 'хан',
 'хангаж',
 'хангалттай',
 'хангасан',
 'хангах',
 'хараад',
 'харах',
 'харж',
 'харилцах',
 'хариу',
 'хариуцлага',
 'харуулж',
 'харьцангуй',
 'хатуу',
 'хийгдэж',
 'хийгээд',
 'хийдэг',
 'хийлээ',
 'хийнэ',
 'хил',
 'хичээл',
 'хол',
 'холбогдох',
 'хамт',
 'холбогдуулан',
 'хамгаалах',
 'хамаагүй',
 'уок',
 'уран',
 'урд',
 'урлаг',
 'урьдчилан',
 'уул',
 'уулзаж',
 'уулзсан',
 'уусан',
 'ухаалаг',
 'учиртай',
 'хаа',
 'хаагчид',
 'хаана',
 'хайртай',
 'халдашгүй',
 'халдвар',
 'халдвараар',
 'халдвараас',
 'халдварт',
 'халдварыг',
 'халдварын',
 'хамгаалалт',
 'холбоотой',
 'холбоотойгоор',
 'хоног',
 'хууль',
 'хуульд',
 'хэвлэл',
 'хэвлэлийн',
 'хэдий',
 'хэзээ',
 'хэлбэл',
 'хэлбэрээр',
 'хэлж',
 'хэлснээр',
 'хэлсэн',
 'хэлье',
 'хэлэлцэн',
 'хэлэлцээд',
 'хэлэх',
 'хэмжээ',
 'хэмжээг',
 'хэмжээнд',
 'хэмжээний',
 'хэмээжээ',
 'хэмээх',
 'хэр',
 'хэргийг',
 'хэргийн',
 'хуулийн',
 'хуулийг',
 'хуулиар',
 'хоногт',
 'хоорондын',
 'хор

In [41]:
trigrams

['халдвараас урьдчилан сэргийлэх',
 'халдварт өвчин судлалын',
 'улсын хурлын гишүүн',
 'хэргийн анхан шатны',
 'улсын онцгой комисст',
 'улсын онцгой комиссын',
 'эрүүгийн хэргийн анхан',
 'эрүүл мэндийн төв',
 'өнгөрсөн долоо хоногт',
 'хүндэтгэн үзэх шалтгаангүйгээр',
 'анхан шатны шүүх',
 'аюулгүй байдлыг хангах',
 'нийслэлийн засаг дарга',
 'нийслэлийн засаг даргын',
 'нийслэлийн прокурорын газраас',
 'монгол улсын засгийн',
 'монгол улсын хурлын',
 'захиргааны хэргийн анхан',
 'дүүргийн засаг даргын',
 'дүүргийн эрүүгийн хэргийн',
 'ерөнхийлөгчийн тамгын газрын']